# Building a Hotel Booking Agent with HeatWave GenAI

In this hands-on lab you will build an **AI-powered hotel booking agent** that can search for hotels, make and cancel reservations, and answer pricing questions by querying PDF invoices — all running inside **MySQL HeatWave** using **LangChain**.

### What you will learn

By the end of this lab you will understand how to:

- Use the **HeatWave GenAI** LLM integration (`MyLLM`) with LangChain agents
- Build **custom tools** that let an agent query and modify a MySQL database
- Implement a **Retrieval-Augmented Generation (RAG)** pipeline backed by HeatWave vector store
- Observe how a ReAct agent **reasons**, selects tools, and chains multiple actions together

### Lab Overview

| Step | What You'll Do |
|------|---------------|
| 1 | Connect to the database |
| 2 | Create a basic agent and attempt to book a hotel |
| 3 | Introspect the `<user>.hotels` table |
| 4 | Build tools for searching, booking, cancelling, and modifying hotels |
| 5 | Build a RAG tool to answer pricing questions from PDF invoices |

> **Prerequisites:** You should have a running MySQL HeatWave instance with MySQL Studio open and a provisioned user account.

---
## Step 1 — Connect to the Database

Start by establishing a connection to your MySQL HeatWave instance. The connection uses the `MYSQL_USER` environment variable that is already configured in MySQL Studio.

If the cell prints `(1,)` you are successfully connected.

In [0]:
import json
import os
import mysql.connector

mydb = mysql.connector.connect(database=os.environ["MYSQL_USER"])

# A quick run to confirm your connection works
cursor = mydb.cursor()
cursor.execute("SELECT 1")
print(cursor.fetchone())

---
## Step 2 — Create an Agent and Try to Book a Hotel

Before building real tools, let's see what happens when an agent **has no real database access**.

The code below does three things:

1. **Initialises the LLM** — `MyLLM` wraps HeatWave's built-in LLM inference so you can use it with LangChain. We bind it to `meta.llama-3.3-70b-instruct`.
2. **Creates a placeholder tool** — LangChain's `initialize_agent` requires at least one tool, so we pass a dummy that always returns `"No real tools available yet."`.
3. **Creates a ReAct agent** — `ZERO_SHOT_REACT_DESCRIPTION` means the agent uses a *Reasoning + Acting* loop: it thinks about what to do, picks a tool, observes the result, and repeats.

In [0]:
from mysql.ai.genai import MyLLM
from langchain_classic.agents.initialize import initialize_agent, AgentType
from langchain_core.tools import Tool

myllm = MyLLM(mydb).bind(model_id="meta.llama-3.3-70b-instruct")

# A dummy tool that does nothing — just so the agent can be created
dummy_tool = Tool(
    name="Placeholder",
    func=lambda x: "No real tools available yet.",
    description="A placeholder tool that does nothing useful.",
)

agent = initialize_agent(
    llm=myllm,
    tools=[dummy_tool],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=2,
    early_stopping_method="generate",
)

Now let's ask it to book a hotel and watch what happens:

In [0]:
agent.invoke({"input": "Book the Hilton Basel for me!"})

**What happened?** The agent has no real tools and no access to your database. It either calls the placeholder (which returns nothing useful) or tries to answer from its own knowledge and **hallucinates**.

This is the core motivation for the rest of the lab: **we need to give the agent real tools** that let it query and modify data in MySQL.

---
## Step 3 — Introspect the `hotels` Table

Before writing tools, let's examine the table you will be working with. Understanding the schema is essential — your tool functions will need to know which columns to query and update.

In [0]:
import pandas as pd

pd.read_sql("SELECT * FROM hotels LIMIT 10", mydb)

**Take note of the following columns — you will use them throughout the lab:**

| Column | Purpose |
|--------|---------|
| `id` | Primary key — your tools will use this to identify hotels |
| `name` | Hotel name (e.g. *Hilton Basel*) |
| `location` | City where the hotel is located |
| `booked` | `1` = booked, `0` = available |
| `checkin_date` / `checkout_date` | Reservation dates |

---
## Step 4 — Build Hotel Search, Booking & Cancellation Tools

Now it is time to give the agent real capabilities. You will create **five tools** using LangChain's `Tool` class.

Each tool is composed of two parts:

1. **A Python function** that performs the actual work (SQL query, database update, etc.).
2. **A `Tool` wrapper** that gives the function a `name` and a `description`. The description is what the LLM reads when deciding which tool to call — so make it clear and specific.

After creating each tool we will **re-create the agent** with the growing toolset and test it with a prompt.

### 4.1 — Search Hotels by Location

Our first real tool lets the agent find hotels in a given city. The function runs a simple `SELECT … WHERE location = ?` query and returns the results as JSON.

> **Why JSON?** The agent's LLM expects text input/output. Converting the pandas DataFrame to JSON gives the model structured data it can reason over.

In [0]:
def search_hotels_by_location(location: str):
    # Remove common meta-commentary patterns as they sometimes
    # appear in the LLM-generated input to this function
    # Split on newlines and take only the first non-empty line
    location = location.split('\n')[0].strip()

    # pd.read_sql executes a SQL statements
    # and stores a result set to a pandas dataframe
    df = pd.read_sql("SELECT * FROM hotels WHERE location = %s", mydb, params=(location,))
    # convert to json that has column names and record values
    response = df.to_json(orient='records')
    return response

# Wrap function into a Tool class
# Provide description passed to an LLM when selecting a tool
search_hotels_by_location_tool = Tool(
    name="Search hotels by location",
    func=search_hotels_by_location,
    description="Search hotels by location.",
)

#### Re-create the agent with the new tool

Every time we add a tool we need to recreate the agent so it knows about the updated toolset.

In [0]:
tools = [
    search_hotels_by_location_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=4,
    early_stopping_method="generate",
)

#### Test it — search for hotels in Basel

Watch the verbose output: you should see the agent **think** about the question, **choose** the `Search hotels by location` tool, pass `"Basel"` as input, and then **use the results** to formulate its answer.

In [0]:
agent.invoke({"input": "Find me hotels in Basel?"})

The agent now successfully queries your MySQL database to answer the question. Compare this to Step 2 where it could only hallucinate!

Let's keep adding more tools to handle additional use cases.

### 4.2 — Search Hotels by Name

Sometimes users ask for a hotel by name rather than city. This tool should use a `LIKE` query for fuzzy matching so that `"Zurich"` matches `"Sheraton Zurich"`, `"Hyatt Regency Zurich"`, etc.

In [0]:
def search_hotels_by_name(name: str):
    # Hint: similar pattern as above, but query the 'name' column
    # Use a LIKE query
    
    #######################
    pass  # your code goes here

search_hotels_by_name_tool = Tool(
    name="Search hotels by name",
    func=search_hotels_by_name,
    description="Search hotels by name.",
)

tools = [
    search_hotels_by_location_tool,
    search_hotels_by_name_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=4,
    early_stopping_method="generate",
)

#### Test it — search by name

In [0]:
agent.invoke({"input": "Find me hotels that have Zurich in their name?"})

Notice how the agent now has **two tools** to choose from and picks the right one based on the user's question. With location-based queries it uses `Search hotels by location`; with name-based queries it uses `Search hotels by name`.

### 4.3 — Book a Hotel

Now let's give the agent the ability to **write** to the database, not just read. This tool sets `booked = 1` for a given hotel ID.

> **Note:** The agent will typically chain tools together — first *searching* for a hotel to find its ID, then *booking* it. Watch the ReAct trace to see this multi-step reasoning in action.

In [0]:

def book_hotel(hotel_id: int):
    # Hint: Obtain a cursor as mydb.cursor() and call execute on it
    
    #########
    pass  # your code goes here

book_hotel_tool = Tool(
    name="Book hotel",
    func=book_hotel,
    description="Book a hotel by its ID. If the hotel is successfully booked, returns a NULL, raises an error if not.",
)

tools = [
    search_hotels_by_location_tool,
    search_hotels_by_name_tool,
    book_hotel_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=4,
    early_stopping_method="generate",
)

#### Test it — book the Hilton Basel

This time the agent should: search for the Hilton Basel → get its ID → call the Book tool.

In [0]:
agent.invoke({"input": "Book the Hilton Basel for me."})

**Verify the booking** — run the query below to confirm `booked = 1` for the Hilton Basel:

In [0]:
pd.read_sql("SELECT * FROM hotels", mydb)

### 4.4 — Cancel a Hotel

The inverse of booking: this tool sets `booked = 0` for a given hotel ID.

In [0]:

def cancel_hotel(hotel_id: int):
    ###############
    pass  # your code goes here

cancel_hotel_tool = Tool(
    name="Cancel hotel",
    func=cancel_hotel,
    description="Cancel a hotel by its ID.",
)

tools = [
    search_hotels_by_location_tool,
    search_hotels_by_name_tool,
    book_hotel_tool,
    cancel_hotel_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=6,
    early_stopping_method="generate",
)

#### Test it — cancel one hotel and book another

This is a great test of multi-step reasoning. The agent needs to: search for Hilton Basel → cancel it → search for Hyatt Regency → book it. Watch the verbose trace!

In [0]:
agent.invoke({"input": "Cancel Hilton Basel and book the Hyatt Regency instead."})

### 4.5 — Update Hotel Dates

Our final CRUD tool lets the agent update check-in and check-out dates for a reservation.

Since LangChain's `Tool` class accepts a single string input, we use a **comma-separated format**: `hotel_id, checkin_date, checkout_date`. The tool description tells the LLM exactly how to format the input.

In [0]:
def update_hotel(input_str: str):
    #############
    pass  # your code goes here

update_hotel_tool = Tool(
    name="Update hotel",
    func=update_hotel,
    description="Update hotel dates. Input: 'hotel_id, checkin_date, checkout_date' (e.g., '7, 2026-04-10, 2026-04-19')",
)


tools = [
    search_hotels_by_location_tool,
    search_hotels_by_name_tool,
    book_hotel_tool,
    cancel_hotel_tool,
    update_hotel_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate",
)

#### Test it — book a hotel with specific dates

The agent should search for the hotel, update the check-in/check-out dates, and then book it — all in one turn.

In [0]:
agent.invoke({
    "input": "Book the Sheraton Zurich hotel. My check-in dates would be from April 10, 2026 to April 15, 2026."})

You now have a fully functional CRUD agent with **five tools**. The agent can search, book, cancel, and update hotel reservations by reasoning over which tools to call and in what order.

In the next step we will add one more powerful capability: **answering questions from unstructured documents** using RAG.

---
## Step 5 — Build a RAG Tool for Invoice Questions

Notice that the `hotels` table contains **no pricing information**. However, there are hotel invoices stored as PDF files in object storage that do contain prices.

In this step you will:
1. Load those PDFs into HeatWave's **vector store** (via the Chats pane)
2. Build a **RAG tool** that retrieves relevant invoice segments so the agent can answer pricing questions

### 5.1 — Load the invoices into HeatWave vector store

Open the following [Pre-authenticated request URL](https://objectstorage.us-chicago-1.oraclecloud.com/p/amCzkNqwKJIQVRpX6LtjKZ0q7Fg2zKbflk66E5idDBOH3S8m06qUsSGEuxG7cjFk/n/mysqlpm/b/amld-workshop/o/hotel_invoices.pdf) and copy the link.

Then:
1. Navigate to the **Chats** pane in MySQL Studio
2. Load the PDF into **HeatWave vector store** using the copied URL
3. Try asking the chatbot a pricing question to confirm the invoices loaded correctly
4. Come back to this notebook and continue below

### 5.2 — Create the RAG Tool

The function below calls the `sys.ML_RAG` stored procedure to retrieve relevant document segments from the vector store:

```sql
CALL sys.ML_RAG("input string", @response, JSON_OBJECT('skip_generate', true))
SELECT @response
```

**Why `skip_generate = true`?** This tells HeatWave to return the raw document segments (citations) **without** generating a final answer. Our LangChain agent will do the reasoning instead — it reads the segments and formulates its own response. This gives us more control over the final output.

In [0]:
import json
import os

from langchain_classic.agents.initialize import initialize_agent, AgentType
from langchain_core.documents import Document
from langchain_core.tools import Tool

from mysql.ai.genai import MyLLM
import mysql.connector


mydb = mysql.connector.connect(database=os.environ["MYSQL_USER"])

myllm = MyLLM(mydb).bind(model_id="meta.llama-3.3-70b-instruct")

def rag(input: str) -> list[Document]:
    # Hint:
    #   1. Call sys.ML_RAG with skip_generate=true to get raw citations
    #   2. Parse the JSON response
    #   3. Return a list of LangChain Document objects with metadata

    #############
    pass  # your code goes here

rag_tool = Tool(
    name="Knowledge Base",
    func=rag,
    description="Useful for looking up hotel invoices.",
)

tools = [
    rag_tool,
]

agent = initialize_agent(
    llm=myllm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate",
)

### 5.3 — Test the RAG Tool

Now ask pricing questions that require information from the PDF invoices.

Watch the verbose output: the agent should call the **Knowledge Base** tool, receive relevant invoice segments, and then reason over them to produce an answer.

In [0]:
agent.invoke({"input": "What is a price of accommodation at Hilton?"})

In [0]:
agent.invoke({"input": "How much does a hotel room cost on average in the city of Bern?"})

---
## Congratulations!

You have built a fully functional hotel booking agent that can:

- **Search** for hotels by name or location
- **Book** and **cancel** reservations
- **Update** check-in and check-out dates
- **Answer pricing questions** by retrieving information from PDF invoices via RAG

All LLM inference ran through **MySQL HeatWave** — no external LLM API calls required.

### Key takeaways

- A **ReAct agent** reasons step-by-step, choosing the right tool for each sub-task and chaining actions together.
- **Tool descriptions matter** — the LLM uses them to decide which tool to call and how to format the input.
- **RAG** lets the agent answer questions from unstructured documents (PDFs, invoices, contracts) without needing that data in a relational table.
- **HeatWave GenAI** provides LLM inference, vector storage, and RAG all within MySQL — simplifying the architecture and keeping data close to the compute.